# 目的

#每日收盤後開始爬股價

#可以視情況條種是否抓營收，大概都隔月的10 後營收才會公布

## 【資料源】
來自 _BaseInfo  _CrawBase 會先去取基本相關資料 (股票號碼...等)

## 【輸出】
股票相關推薦圖表/數據

In [1]:
import pandas as pd

#  Update Base Info

In [2]:
 
#%run _BaseInfoETF.ipynb
%run _CrawBase.ipynb
#%run _LineMsg.ipynb
%run _genHtml.ipynb

C:\Users\user\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


%run RunRealTimeStock.ipynb

In [3]:
def MainStock(stock_number, start_month,end_time=datetime.now().strftime("%Y-%m-%d"),updateRevenue=False):  

    strmsg ='【stock_number】'+stock_number+'\n' +'【start_month】'+start_month+'\n'+'【end_time】'+end_time+'\n' 
    try:
        #處理 Stock
        RowData_df_craw_stock,His_Stock ,isSuccess= craw_stock(stock_number,start_month,end_time)
        #data_final,df_final=DataCleaning(RowData_df_craw_stock)
        print("【craw_stock】 success")
        
        #處理 Revenue
        if(updateRevenue):
            RowData_df_craw_revenue,His_Stock=craw_revenue(stock_number)
            df_Revenue=RevenueDataCleaning(RowData_df_craw_revenue)
            print("【craw_revenue】 success")
            
        #處理相關指標轉成html 
        stock_data=data_process(RowData_df_craw_stock)
    
        level,interval_type, lower_bound, upper_bound=get_interval(stock_data.iloc[-1]['收盤價'],np.array(stock_data['收盤價']))
        stock_data['stock_number']=stock_number
        stock_data['level']=level
        stock_data['interval_type']=interval_type
        stock_data['lower_bound']=lower_bound
        stock_data['upper_bound']=upper_bound
        save_df_to_excel(stock_data)
        gen_html(stock_number,stock_data)

        #sendLineMessage('成功'+'\n'+strmsg )
    except:
        #sendLineMessage('失败'+'\n'+strmsg)
        print('MainStock Fail'+'\n'+strmsg)
        
    #return data_final,df_final,df_Revenue
    

# 清資料使用
type_='Stock_His'
strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
df=get_DB()
df[~((df['status'] != 'OK')& (df['status'] != '很抱歉，沒有符合條件的資料!'))].to_csv('BaseInfo/His_Stock.csv',index=False) 
DDD=df[((df['status'] != 'OK')& (df['status'] != '很抱歉，沒有符合條件的資料!'))]['stock_number'].unique()

start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False
file_path = 'D:\Project\Jupyter\Stock\Main\DailyResultLast.xlsx'   

stock_number=str(6763)
if (want_update_Stock):        
    type_='Stock_His'
    strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
    delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_)
try:
    if(want_update_Revenue):
        MainStock(stock_number,start_month,want_update_Revenue)
    else:
        MainStock(stock_number,start_month)
except Exception as error:
    print("An exception occurred:", error)
    print('Action Fail'+'\n')

start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False


for code in _baseInfo.dropna()['公司代號']:#.iloc[::-1]:
    stock_number=str(code)
    if (want_update_Stock):        
        type_='Stock_His'
        strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
        delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_)
    try:
        if(want_update_Revenue):
            MainStock(stock_number,start_month,want_update_Revenue)
        else:
            MainStock(stock_number,start_month)
    except Exception as error:
        print("An exception occurred:", error)
        print('Action Fail'+'\n')

In [ ]:
import threading
from datetime import datetime, timedelta

start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False

# 定義一個處理股票資料的函數，接收股票代號列表作為參數
def process_stock_codes(stock_codes):
    # 計算月份字串（僅需計算一次）
    strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')

    for code in stock_codes:
        stock_number = str(code)
        try:
            # 如果要更新歷史資料
            if want_update_Stock:        
                type_ = 'Stock_His'
                delete_DB_stock_by_Month_Stock_number(strMonth, stock_number, type_)
        except Exception as error:
            print(f"An exception occurred with stock {stock_number}: {error}")
            print('Action Fail\n')

        try:
            # 根據 want_update_Revenue 的值來決定傳入的參數
            if want_update_Revenue:
                MainStock(stock_number, start_month, want_update_Revenue)
            else:
                MainStock(stock_number, start_month)
        except Exception as error:
            print(f"An exception occurred with stock {stock_number}: {error}")
            print('Action Fail\n')

            
            
            
# 分割資料，取前 500 個和其餘的股票代號

first_500_codes = _baseInfo[_baseInfo['Type']=='上市'].dropna()['公司代號']
remaining_codes =_baseInfo[_baseInfo['Type']!='上市'].dropna()['公司代號']

# 創建兩個執行緒，分別處理前 500 個和其餘股票代號
thread1 = threading.Thread(target=process_stock_codes, args=(first_500_codes,))
thread2 = threading.Thread(target=process_stock_codes, args=(remaining_codes,))

# 啟動執行緒
thread1.start()
thread2.start()

# 等待兩個執行緒完成
thread1.join()
thread2.join()

print("Stock processing completed.")

【craw_stock】 success
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-132.xlsx
log/DailyResult/DailyResult_2024-11-13.xlsx
